In [1]:
import numpy as np
import pandas as pd
import re
import string

import spacy 
import sklearn

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
from nltk.corpus import stopwords

In [2]:
nlp = spacy.load('en_core_web_lg')

In [3]:
book_list = pd.read_pickle('../data/scary_books.pkl')

In [4]:
book_list.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2063 entries, 0 to 2062
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   title             2063 non-null   object 
 1   author            2063 non-null   object 
 2   description       2063 non-null   object 
 3   publisher         1766 non-null   object 
 4   customer_reviews  2042 non-null   float64
 5   stars             2042 non-null   float64
 6   url               2063 non-null   object 
dtypes: float64(2), object(5)
memory usage: 112.9+ KB


In [5]:

book_list.isnull().sum()

title                 0
author                0
description           0
publisher           297
customer_reviews     21
stars                21
url                   0
dtype: int64

In [6]:
book_list['stars']=book_list['stars'].fillna(book_list['stars'].mean())

In [7]:
book_list['customer_reviews']=book_list['customer_reviews'].fillna(0)

In [8]:
book_list['publisher']=book_list['publisher'].fillna('Self')

In [9]:
book_list["id"] = book_list.index + 1

In [10]:
book_list.head()

,title,author,description,publisher,customer_reviews,stars,url,id
0,A Time to Kill: A Novel [Kindle Edition],John Grisham,"Don’t miss a sneak peek of Sycamore Row, John ...",Dell,918.0,4.5,http://www.amazon.com/A-Time-Kill-Novel-ebook/...,1
1,Failure to Appear (J. P. Beaumont Novel) [Kind...,J. A. Jance,A shattering tale of tragedy and horror from t...,William Morrow; Reprint edition,34.0,4.8,http://www.amazon.com/Failure-Appear-J-Beaumon...,2
2,Twelve Years a Slave [Kindle Edition],Solomon Northup,"First published in 1853, Twelve Years a Slave ...",HarperPerennial Classics,1374.0,4.6,http://www.amazon.com/Twelve-Years-Slave-Solom...,3
3,Without Due Process (J. P. Beaumont Novel) [Ki...,J. A. Jance,Top ten New York Times bestselling author J.A....,William Morrow; Reissue edition,39.0,4.8,http://www.amazon.com/Without-Due-Process-Beau...,4
4,Devil's Claw: A Joanna Brady Mystery (Joanna B...,J. A. Jance,In the night-still canyons of the Arizona dese...,William Morrow; Reissue edition,63.0,4.5,http://www.amazon.com/Devils-Claw-Joanna-Myste...,5


In [11]:
book_list["text"] = book_list["title"].map(str) + " " + book_list["author"] +" "+ book_list["description"]+ " "+book_list['publisher']
book_list.head(2)

,title,author,description,publisher,customer_reviews,stars,url,id,text
0,A Time to Kill: A Novel [Kindle Edition],John Grisham,"Don’t miss a sneak peek of Sycamore Row, John ...",Dell,918.0,4.5,http://www.amazon.com/A-Time-Kill-Novel-ebook/...,1,A Time to Kill: A Novel [Kindle Edition] John ...
1,Failure to Appear (J. P. Beaumont Novel) [Kind...,J. A. Jance,A shattering tale of tragedy and horror from t...,William Morrow; Reprint edition,34.0,4.8,http://www.amazon.com/Failure-Appear-J-Beaumon...,2,Failure to Appear (J. P. Beaumont Novel) [Kind...


In [12]:
book_list_df = book_list[['id', 'text', 'title', 'author', 'url']]

In [13]:
book_list_df.head()

,id,text,title,author,url
0,1,A Time to Kill: A Novel [Kindle Edition] John ...,A Time to Kill: A Novel [Kindle Edition],John Grisham,http://www.amazon.com/A-Time-Kill-Novel-ebook/...
1,2,Failure to Appear (J. P. Beaumont Novel) [Kind...,Failure to Appear (J. P. Beaumont Novel) [Kind...,J. A. Jance,http://www.amazon.com/Failure-Appear-J-Beaumon...
2,3,Twelve Years a Slave [Kindle Edition] Solomon ...,Twelve Years a Slave [Kindle Edition],Solomon Northup,http://www.amazon.com/Twelve-Years-Slave-Solom...
3,4,Without Due Process (J. P. Beaumont Novel) [Ki...,Without Due Process (J. P. Beaumont Novel) [Ki...,J. A. Jance,http://www.amazon.com/Without-Due-Process-Beau...
4,5,Devil's Claw: A Joanna Brady Mystery (Joanna B...,Devil's Claw: A Joanna Brady Mystery (Joanna B...,J. A. Jance,http://www.amazon.com/Devils-Claw-Joanna-Myste...


In [14]:
book_list_df.shape

(2063, 5)

In [15]:

stop = stopwords.words('english')
stop_words_ = set(stopwords.words('english'))
wn = WordNetLemmatizer()


In [16]:

def black_txt(token):
    return  token not in stop_words_ and token not in list(string.punctuation)  and len(token)>2   

Remove stop Words, Remove non-alphanumeric characters, lemmatize columns, merge all columns in order to create a corpus of text for each book

In [17]:
def clean_txt(text):
    clean_text = []
    clean_text2 = []
    text = re.sub("'", "",text)
    text = re.sub("(\\d|\\W)+"," ",text)
    text = text.replace("kindle", "")
    text = text.replace("edition", "")
    clean_text = [ wn.lemmatize(word, pos="v") for word in word_tokenize(text.lower()) if black_txt(word)]
    clean_text2 = [word for word in clean_text if black_txt(word)]
    return " ".join(clean_text2)

In [18]:
book_list_df['text'] = book_list_df['text'].apply(clean_txt)

<ipython-input-18-f10be6f7747f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  book_list_df['text'] = book_list_df['text'].apply(clean_txt)


In [19]:
book_list_df.head()

,id,text,title,author,url
0,1,time kill novel kindle edition john grisham mi...,A Time to Kill: A Novel [Kindle Edition],John Grisham,http://www.amazon.com/A-Time-Kill-Novel-ebook/...
1,2,failure appear beaumont novel kindle edition j...,Failure to Appear (J. P. Beaumont Novel) [Kind...,J. A. Jance,http://www.amazon.com/Failure-Appear-J-Beaumon...
2,3,twelve years slave kindle edition solomon nort...,Twelve Years a Slave [Kindle Edition],Solomon Northup,http://www.amazon.com/Twelve-Years-Slave-Solom...
3,4,without due process beaumont novel kindle edit...,Without Due Process (J. P. Beaumont Novel) [Ki...,J. A. Jance,http://www.amazon.com/Without-Due-Process-Beau...
4,5,devil claw joanna brady mystery joanna brady m...,Devil's Claw: A Joanna Brady Mystery (Joanna B...,J. A. Jance,http://www.amazon.com/Devils-Claw-Joanna-Myste...


Speculative word bank, hand-built, with inspiration from the Word2Vec model I built in the previous capstone. 

In [20]:
reader_choices = ['monster', 'lore', 'grave', 'animal', 'dread', 'haunted', 'claw', 'eerie', 'old', 'mystery', 'dark', 'fear', 'ghost', 'terror', 'blood', 'gore', 'death', 'danger', 'atmosphere', 'supernatural']

In [21]:
import random

In [22]:

sample_reader = random.sample(reader_choices, 3)

In [23]:
sample_reader = ' '.join([str(elem) for elem in sample_reader])

In [24]:
sample_reader

'animal old fear'

In [25]:
reader_number = 1

Different from a collaborative filtering approach which builds a model from a user's past behaviors, a Content based approach utilizes a series of discrete characteristics of an item in order to recommend additional items with similar properties. My implementation could be improved by adding data collectino features that would eventually turn it in to a hybrid approach. 

In [26]:
dict = {'user_id': [reader_number], 'selections': [sample_reader]}

In [27]:
user_df = pd.DataFrame(dict)

In [28]:
user_df

,user_id,selections
0,1,animal old fear


In [29]:
user_df['text'] = user_df['selections'].map(str).apply(clean_txt)

In [30]:
user_df

,user_id,selections,text
0,1,animal old fear,animal old fear


In [31]:
user_recc_df = user_df[['user_id', 'text']]

In [32]:
u = 1
index = np.where(user_recc_df['user_id']==u)[0][0]
user_q = user_recc_df.iloc[[index]]
user_q

,user_id,text
0,1,animal old fear


In [70]:
#initializing tfidf vectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()

tfidf_bookid = tfidf_vectorizer.fit_transform((book_list_df['text'])) #fitting and transforming the vector
tfidf_bookid

<2063x25213 sparse matrix of type '<class 'numpy.float64'>'
	with 242641 stored elements in Compressed Sparse Row format>

Using TF-IDF

In [37]:
from sklearn.metrics.pairwise import cosine_similarity

user_tfidf = tfidf_vectorizer.transform(user_q['text'])

cos_similarity_tfidf = map(lambda x: cosine_similarity(user_tfidf, x),tfidf_bookid)

In [38]:
output2 = list(cos_similarity_tfidf)

In [49]:
def get_recommendation(top, book_list_df, scores):
  recommendation = pd.DataFrame(columns = ['title', 'score'])
  count = 0
  for i in top:
      recommendation.at[count, 'user_id'] = u
      recommendation.at[count, 'id'] = book_list_df['id'][i]
      recommendation.at[count, 'title'] = book_list_df['title'][i]
      recommendation.at[count, 'author'] = book_list_df['author'][i]
      recommendation.at[count, 'score'] =  scores[count]
      count += 1
  return recommendation

In [73]:
top = sorted(range(len(output2)), key=lambda i: output2[i], reverse=True)[:10]
list_scores = [output2[i][0][0] for i in top]
get_recommendation(top,book_list_df, list_scores)

,title,score,user_id,id,author
0,School of the Supernatural: Live the Supernatu...,0.136929,1.0,1029.0,Ryan Wyatt
1,Unlocking Heaven: Keys to Living Naturally Sup...,0.110197,1.0,810.0,Kevin Dedmon
2,Splendour Falls [Kindle Edition],0.104865,1.0,789.0,Susanna Kearsley
3,The Venus Deal: A California Century Mystery #...,0.104545,1.0,684.0,Ken Kuhlken
4,The Old Man's Back in Town (SHORT STORY from t...,0.103227,1.0,655.0,Ann Charles
5,How to Walk In the Supernatural Power of God [...,0.099292,1.0,882.0,Guillermo Maldonado
6,Drummer Boy: A Supernatural Thriller [Kindle E...,0.098258,1.0,205.0,Scott Nicholson
7,Ethereal Messenger: Three Novels [Kindle Edition],0.092454,1.0,952.0,Scott Nicholson
8,The Ultimate Mystery Thriller Horror Box Set (...,0.091466,1.0,46.0,Cathy Perkins
9,Twisted Vengeance: A Supernatural Thriller [Ki...,0.089446,1.0,876.0,Jeff Bennington


Using spaCy

In [74]:
%%time
list_docs = []
for i in range(len(book_list_df)):
  doc = nlp("u'" + book_list_df['text'][i] + "'")
  list_docs.append((doc,i))
print(len(list_docs))

2063
CPU times: user 44.2 s, sys: 4.58 s, total: 48.8 s
Wall time: 49.1 s


In [75]:
def calculateSimWithSpaCy(nlp, df, user_text, n=6):
    # Calculate similarity using spaCy
    list_sim =[]
    doc1 = nlp("u'" + user_text + "'")
    for i in df.index:
      try:
            doc2 = list_docs[i][0]
            score = doc1.similarity(doc2)
            list_sim.append((doc1, doc2, list_docs[i][1],score))
      except:
        continue

    return  list_sim

In [76]:
user_q.text[0]

'animal gore terror'

In [77]:
%%time
 df3 = calculateSimWithSpaCy(nlp, book_list_df, user_q.text[0], n=15)

CPU times: user 1.44 s, sys: 15.6 ms, total: 1.46 s
Wall time: 1.46 s


In [78]:
df_recom_spacy = pd.DataFrame(df3).sort_values([3], ascending=False).head(10)

In [79]:

df_recom_spacy.reset_index(inplace=True)

In [80]:
index_spacy = df_recom_spacy[2]
list_scores = df_recom_spacy[3]

In [81]:
get_recommendation(index_spacy, book_list_df, list_scores)

,title,score,user_id,id,author
0,Crime Scene: True Stories of Crime and Detecti...,0.643786,1.0,446.0,Gary C. King
1,HORRORIFIC 3 Stories Even Colder and Darker [K...,0.630761,1.0,695.0,Gene-Michael Higney
2,Jack in the Box (Shadow War Thriller) [Kindle ...,0.618854,1.0,1226.0,John Weisman
3,Mob Killer: [Kindle Edition],0.615219,1.0,233.0,Anthony M. DeStefano
4,The Dead Room [Kindle Edition],0.61414,1.0,1945.0,Robert Ellis
5,Deadeye Dick: A Novel [Kindle Edition],0.611976,1.0,130.0,Kurt Vonnegut
6,Blood at the Root (Inspector Banks Novels) [Ki...,0.611266,1.0,1477.0,Peter Robinson
7,They're Coming For You 2: More Scary Stories t...,0.609444,1.0,638.0,O. Penn-Coughin
8,Die Trying: A Zombie Apocalypse [Kindle Edition],0.608949,1.0,149.0,Nicholas Ryan
9,Sixty-Five Stirrup Iron Road [Kindle Edition],0.608672,1.0,579.0,Brian Keene
